In [ ]:
import os
import sys
import joblib
sys.path.insert(1, '/mnt/d/PowerTAC/Python/python_utils/helper') 

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

from statistics import mean
from matplotlib import pyplot as plt
from plotly.subplots import make_subplots
from read_mongo_collection import HelperToReadMongo

from tensorflow.keras.models import load_model
from sklearn.metrics import mean_squared_error, mean_absolute_error

sns.set_theme(style="darkgrid")

# See complete data
pd.set_option('max_columns', None)
pd.set_option("max_rows", None)
np.set_printoptions(threshold=np.inf)

In [ ]:
helper_mongo = HelperToReadMongo()

In [ ]:
# Specify names of all the databases here
database = 'PowerTAC2021_Trials2'

# Specify the storage path along with directory
storage_path = '/mnt/d/PowerTAC/PowerTAC2021/experiments/capacity_transaction_check/peak_prediction_check/PowerTAC2021_Trials2_test2/'
os.makedirs(storage_path, exist_ok=True) 

# Specify list of the collections to be analysed here
list_of_collections = helper_mongo.connect_to_mongo(database).list_collection_names()

# Keep all the collections at one place
dict_of_dataframes = dict()

In [ ]:
for collection in list_of_collections:

    db = helper_mongo.query_to_mongo(database, collection, server_ip='<ip>', ssh_username='<username>', ssh_password='<password>', remote=False)
        
    dict_of_dataframes.update({collection: db})

In [ ]:
dict_of_dataframes.keys()

In [ ]:
###############################################
############ Plot using Plotly ################
###############################################
    
def capacity_transaction_peak_prediction(game):    
        
    fig = go.Figure()
                            
    database_nb = dict_of_dataframes.get('Capacity_Transaction_Net_Demand_Info')
    database_ct = dict_of_dataframes.get('CapacityTransaction_Info')
    database_pp = dict_of_dataframes.get('Predicted_Peaks')    
    db1 = dict_of_dataframes.get('Predicted_demands')

    db_nb = database_nb[database_nb['Game_Name'] == game]
    db_ct = database_ct[database_ct['Game_Name'] == game]
    db_pp = database_pp[database_pp['Game_Name'] == game]
    db1 = db1[db1['Game_Name'] == game]
    
    db1["Predicted Net Demand"] = db1['Predicted Consumption'] - db1['Predicted Production']

    fig.add_trace(go.Scatter(x = db_nb['Timeslot'],
                                y = db_nb['Mean'], 
                                name = 'Mean Demand',
                               ))

    fig.add_trace(go.Scatter(x = db_nb['Timeslot'],
                                y = db_nb['Threshold'], 
                                name = 'Threshold',
                               ))

    fig.add_trace(go.Scatter(x = db_nb['Timeslot'],
                                y = (db_nb['Mean']+2*db_nb['Stdev']), 
                                name = 'mu + 2sigma',
                               ))

    fig.add_trace(go.Scatter(x = db_nb['Timeslot'],
                                y = db_nb['Demand'], 
                                name = 'Net Demand',
                                line=dict(color='mediumslateblue')
                             ))
    
    fig.add_trace(go.Scatter(x = db1['Timeslot'],
                                y = db1['Predicted Net Demand'], 
                                name = 'Predicted Net Demand',
                                line=dict(color='tomato')
                               ))
    
    fig.add_trace(go.Scatter(x = db_ct['Peak_Timeslot'],
                             y = (db_ct['Threshold'] + db_ct['Exceeded_MWh']),
                             name = 'Peak Timeslots',
                             mode = 'markers',
                             marker=dict(color='red', size=12),
                            ))

    db_final = pd.merge(left=db_nb, right=db_pp, how='inner', on=['Game_Name', 'Timeslot'])
    
    db_final['out'] = db_final['Prediction']*db_final['Demand']
    db_final =  db_final[db_final['out'] != 0]
        
    fig.add_trace(go.Scatter(x = db_final['Timeslot'],
                             y = db_final['out'],
                             name = 'Predicted Peak',
                             mode = 'markers',
                             marker=dict(color='black', size=8),
                            ))
    
    for value in db_ct['Timeslot'].unique():
        fig.add_vline(x=value, line_width=3, line_dash="dash", line_color="green")
                    
    fig.update_yaxes(title_text="Quantity (KWh)")
    fig.update_xaxes(title_text="Timeslot")
    
    fig.update_layout(
        title='capacity_transaction',
        height=750, 
        width=1550, 
        template='ggplot2',
        hovermode="x"
    )
    
    for i in fig['layout']['annotations']:
        i['font'] = dict(size=14)
    
    path = storage_path + game
    os.makedirs(path, exist_ok=True) 
    fig.write_html(path + "/capacity_transaction.html")

In [ ]:
###############################################
############ Plot using Plotly ################
###############################################
    
def capacity_transaction_peak_distribution(game):    
        
    fig = go.Figure()

    database_ct = dict_of_dataframes.get('CapacityTransaction_Info')
    database_cal = dict_of_dataframes.get('Calendar_Info')

    db_ct = database_ct[database_ct['Game_Name'] == game]
    db_cal = database_cal[database_cal['Game_Name'] == game]

    db_final = pd.merge(left=db_ct, right=db_cal, how='inner', left_on=['Game_Name', 'Peak_Timeslot'], right_on=['Game_Name', 'Timeslot'])
    db_final['Hour_of_Week'] = db_final['Day_of_Week'] * db_final['Hour_of_Day']
    db_final = db_final[['Hour_of_Week', 'Peak_Timeslot']]
    db_final = db_final.groupby('Hour_of_Week')
    group_db = db_final.count()

    fig.add_trace(go.Bar(x = list(db_final.groups.keys()),
                             y = group_db['Peak_Timeslot'].tolist(),
                             marker_color='darkorchid',
                            ))

    fig.update_xaxes(range=[0,168], title_text="Hour of Weeks")
    fig.update_yaxes(range=[0,6], title_text="Number of Peaks")

    fig.update_layout(
        height=750, 
        width=1550, 
        template='ggplot2',
        hovermode="x"
    )

    for i in fig['layout']['annotations']:
        i['font'] = dict(size=14)

    path = storage_path + game
    os.makedirs(path, exist_ok=True) 
    fig.write_html(path + "/capacity_transaction_peak_distribution.html")

In [ ]:
def net_demand_error_plot(game):
    
    db = dict_of_dataframes.get('DistributionTransaction_and_Report_Info').copy(deep=True)
    db = db[db['Game_Name'] == game]
    
    db1 = dict_of_dataframes.get('Predicted_demands').copy(deep=True)
    db1 = db1[db1['Game_Name'] == game]
    
    fig = go.Figure()
    
    fig.add_trace(go.Scatter(x = db['Timeslot'],
                                y = db['Total_Consumption'], 
                                name = 'Actual Consumption',
                                line=dict(color='mediumslateblue')
                               ))
    
    fig.add_trace(go.Scatter(x = db1['Timeslot'],
                                y = db1['Predicted Consumption'], 
                                name = 'Predicted Consumption',
                                line=dict(color='tomato')
                               ))
    
    fig.update_yaxes(title_text="Quantity (KWh)")
    fig.update_xaxes(title_text="Timeslot")
    
    fig.update_layout(
        title='Consumption Prediction Error-plot',
        height=750, 
        width=1550, 
        template='ggplot2',
        hovermode="x"
    )
    
    for i in fig['layout']['annotations']:
        i['font'] = dict(size=14)
    
    path = storage_path + game
    os.makedirs(path, exist_ok=True) 
    fig.write_html(path + "/consumption_error_plot.html")
    
    fig = go.Figure()
    
    fig.add_trace(go.Scatter(x = db['Timeslot'],
                                y = db['Total_Production'], 
                                name = 'Actual Production',
                                line=dict(color='mediumslateblue')
                               ))
    
    fig.add_trace(go.Scatter(x = db1['Timeslot'],
                                y = db1['Predicted Production'], 
                                name = 'Predicted Production',
                                line=dict(color='tomato')
                               ))
    
    fig.update_yaxes(title_text="Quantity (KWh)")
    fig.update_xaxes(title_text="Timeslot")
    
    fig.update_layout(
        title='Production Prediction Error-plot',
        height=750, 
        width=1550, 
        template='ggplot2',
        hovermode="x"
    )
    
    for i in fig['layout']['annotations']:
        i['font'] = dict(size=14)
    
    path = storage_path + game
    os.makedirs(path, exist_ok=True) 
    fig.write_html(path + "/production_error_plot.html")
    
    db["Net_Demand"] = db['Total_Consumption'] - db['Total_Production']
    db1["Predicted Net Demand"] = db1['Predicted Consumption'] - db1['Predicted Production']
    
    fig = go.Figure()
    
    fig.add_trace(go.Scatter(x = db['Timeslot'],
                                y = db['Net_Demand'], 
                                name = 'Actual Net Demand',
                                line=dict(color='mediumslateblue')
                               ))
    
    fig.add_trace(go.Scatter(x = db1['Timeslot'],
                                y = db1['Predicted Net Demand'], 
                                name = 'Predicted Net Demand',
                                line=dict(color='tomato')
                               ))
    
    fig.update_yaxes(title_text="Quantity (KWh)")
    fig.update_xaxes(title_text="Timeslot")
    
    fig.update_layout(
        title='Net Demand Prediction Error-plot',
        height=750, 
        width=1550, 
        template='ggplot2',
        hovermode="x"
    )
    
    for i in fig['layout']['annotations']:
        i['font'] = dict(size=14)
    
    path = storage_path + game
    os.makedirs(path, exist_ok=True) 
    fig.write_html(path + "/net_demand_error_plot.html")

In [ ]:
test_database = dict_of_dataframes.get('Capacity_Transaction_Net_Demand_Info').copy(deep=True)
games = test_database.Game_Name.unique()

In [ ]:
###############################################
############### Function Call #################
###############################################    

for game in games:
        
    try:
        capacity_transaction_peak_prediction(game)
        net_demand_error_plot(game)
    except:
        print("error in ", game)

In [ ]:
fig = go.Figure()

db_ct = dict_of_dataframes.get('CapacityTransaction_Info')
db_cal = dict_of_dataframes.get('Calendar_Info')

db_final = pd.merge(left=db_ct, right=db_cal, how='inner', left_on=['Game_Name', 'Peak_Timeslot'], right_on=['Game_Name', 'Timeslot'])
db_final['Hour_of_Week'] = db_final['Day_of_Week'] * db_final['Hour_of_Day']
db_final = db_final[['Hour_of_Week', 'Peak_Timeslot']]
db_final = db_final.groupby('Hour_of_Week')
group_db = db_final.count()

fig.add_trace(go.Bar(x = list(db_final.groups.keys()),
                         y = group_db['Peak_Timeslot'].tolist(),
                         marker_color='darkorchid',
                        ))

fig.update_xaxes(range=[0,168], title_text="Hour of Weeks")
fig.update_yaxes(title_text="Number of Peaks")

fig.update_layout(
    height=750, 
    width=1550, 
    template='ggplot2',
    hovermode="x"
)

for i in fig['layout']['annotations']:
    i['font'] = dict(size=14)

fig.write_html(storage_path + "/capacity_transaction_peak_distribution.html")